<!--TABLE OF CONTENTS-->
# Table of Contents:
- [Load the Data and Prepare the Data](#Load-the-data)
- [Transform the Data](#Run-Data-Transformation)
- [Analyze the Data](#Analyze)
- [Export the Analyzed Result](#Export-Output)
- [Summary](#End)

## Load the Data and Prepare the Data

In [79]:
import pandas as pd
from datetime import datetime

#### Loan table

In [103]:
loan_payment = pd.read_csv("data/Loan_Data - Sheet1.csv")

# rename column names for consistency across tables and reflect output result
loan_payment.rename(columns={"Payment_frequency": "payment_frequency", "Maturity_date": "maturity_date"}, inplace=True)

# convert datetime fields to datetime before creating loan table schema
loan_payment.Date_of_release = pd.to_datetime(loan_payment.Date_of_release)
loan_payment.maturity_date = pd.to_datetime(loan_payment.maturity_date, errors='coerce')
loan_payment.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,payment_frequency,maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,NaT
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


#### Borrower table

In [83]:
borrower_data = pd.read_csv("data/Borrower_Data - Sheet1.csv")

# rename borrower_id for consistency across tables
borrower_data.rename(columns={"Borrower_Id": "Borrower_id", "City": "city"}, inplace=True)
borrower_data.head()

,Borrower_id,State,city,zip code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


#### loan_payment (repayment) table

In [71]:
# correct column names
repayment_data = pd.read_csv("data/Repayment_Data - Sheet1.csv")

# rename mis-placed column names
repayment_data.rename(columns={"Amount_paid": "Date_paid", 
                               "Date_paid": "Amount_paid", 
                               "loan_id(fk)": "loan_id",
                                "payment_id(pk)": "payment_id"}, inplace=True)

# convert to datetime before creating repayment schema
repayment_data.Date_paid = pd.to_datetime(repayment_data.Date_paid)
repayment_data.head()

,loan_id,payment_id,Date_paid,Amount_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


#### Payment_schedule table

In [48]:
payment_schedule = pd.read_csv("data/Schedule_Data.xlsx - Sheet1.csv")

# convert to datetime before creating payment schedule schema
payment_schedule.Expected_payment_date = pd.to_datetime(payment_schedule.Expected_payment_date)
payment_schedule.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333


  ## Transformation

In [104]:
# merge loan and borrower table
Transformation_1 = loan_payment.merge(borrower_data, on='Borrower_id')

In [61]:
Transformation_1
None

In [105]:
Transformation_2 = Transformation_1.merge(payment_schedule, on='loan_id', how='left')

In [75]:
Transformation_2.head()
None

In [106]:
Final_transformation = Transformation_2.merge(repayment_data, on='loan_id', how='left')
Final_transformation.head()
None

## Analyze the Data

In [107]:
# create the current_days_past_due column
Final_transformation['current_days_past_due'] = (datetime.now() - Final_transformation['Expected_payment_date']).dt.days

In [108]:
# apply groupby to filter out distinctive values
grouped = Final_transformation.groupby(['city', 'zip code', 'payment_frequency', 'maturity_date'])

In [112]:
# Calculate last_due_date, last_repayment_date and amount_at_risk and aggregate to grouped data
result = grouped.agg(
    last_due_date=pd.NamedAgg(column='Expected_payment_date', aggfunc='max'),
    last_repayment_date=pd.NamedAgg(column='Date_paid', aggfunc='max'),
    amount_at_risk=pd.NamedAgg(column='Expected_payment_amount', aggfunc=lambda x: x[Final_transformation['Expected_payment_date'] <= datetime.now()].sum())
)
result.reset_index(inplace=True)

### Export Analyzed Result in Excel format

In [116]:
result.head() 

,city,zip code,payment_frequency,maturity_date,last_due_date,last_repayment_date,amount_at_risk
0,12olki,19473,1.007903e+05,2022-01-31,2022-01-27,2022-01-27,1.451381e+07
1,12olki,19473,1.271804e+06,2024-04-30,2024-04-16,2024-04-16,1.373548e+09
2,89hfg,923475,1.029075e+05,2022-03-31,2022-03-26,2022-03-26,1.481868e+07


In [118]:
result.to_excel('output_result.xls', index=False)

In [117]:
result.to_csv('output_result.csv', index=False)